IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\M1\FAULT_M1_5


In [3]:
guasto = 1

VOLO M1 FAULT 5%

In [4]:
rcou_m1_fault5 = pd.read_csv("RCOU.csv")
rcou_m1_fault5 = rcou_m1_fault5.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m1_fault5 = rcou_m1_fault5[((rcou_m1_fault5['C9'] >= potenza) & (rcou_m1_fault5['C10'] >= potenza) & (rcou_m1_fault5['C11'] >= potenza) & (rcou_m1_fault5['C12'] >= potenza) & (rcou_m1_fault5['C13'] >= potenza) & (rcou_m1_fault5['C14']>= potenza))]

rcou_m1_fault5 = rcou_m1_fault5.reset_index(drop=True)

display(rcou_m1_fault5)

,TimeUS,C9,C10,C11,C12,C13,C14
0,192480740,1373,1423,1430,1365,1401,1395
1,192545491,1426,1445,1478,1390,1467,1403
2,192576646,1355,1538,1509,1394,1416,1490
3,192587140,1412,1504,1504,1412,1456,1465
4,192597516,1523,1403,1461,1472,1514,1414
...,...,...,...,...,...,...,...
7818,284579056,1623,1404,1545,1500,1468,1573
7819,284589530,1586,1441,1553,1480,1440,1586
7820,284599918,1557,1458,1561,1453,1425,1583
7821,284612550,1560,1427,1576,1407,1464,1529


In [5]:
min=rcou_m1_fault5['TimeUS'][0]
max=rcou_m1_fault5['TimeUS'][len(rcou_m1_fault5 )-1]
print(max)
print(min)

284622968
192480740


In [6]:
xkf1_m1_fault5 = pd.read_csv("XKF1_0.csv")
xkf1_m1_fault5 = xkf1_m1_fault5.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m1_fault5 = xkf1_m1_fault5[((xkf1_m1_fault5['TimeUS'] >= min) & (xkf1_m1_fault5['TimeUS'] <= max))]
xkf1_m1_fault5 = xkf1_m1_fault5.reset_index(drop=True)
print(xkf1_m1_fault5)

         TimeUS  Roll  Pitch    Yaw
0     192488698  1.25   0.19  19.58
1     192499420  1.34   0.27  19.60
2     192510040  1.42   0.24  19.58
3     192521148  1.48   0.17  19.57
4     192532036  1.47   0.11  19.56
...         ...   ...    ...    ...
8332  284576961  1.35   0.18  19.19
8333  284586972  1.39   0.11  19.12
8334  284597792  1.44   0.02  19.05
8335  284610260  1.63  -0.17  18.93
8336  284620781  1.75  -0.32  18.85

[8337 rows x 4 columns]


In [7]:
att_m1_fault5 = pd.read_csv("ATT.csv")
att_m1_fault5 = att_m1_fault5.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m1_fault5 = att_m1_fault5[((att_m1_fault5['TimeUS'] >= min) & (att_m1_fault5['TimeUS'] <= max))]
att_m1_fault5=att_m1_fault5.reset_index(drop=True)
print(att_m1_fault5)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     192490624    -0.06  1.25      0.00   0.19   19.50  19.58
1     192500805    -0.07  1.34      0.00   0.27   19.50  19.60
2     192512203    -0.08  1.42      0.00   0.24   19.50  19.58
3     192523184    -0.08  1.48      0.00   0.17   19.50  19.57
4     192533737    -0.09  1.47      0.00   0.11   19.50  19.56
...         ...      ...   ...       ...    ...     ...    ...
8337  284578343     4.71  1.35     -2.01   0.18   16.97  19.19
8338  284588791     4.69  1.39     -1.96   0.11   16.98  19.12
8339  284599135     4.67  1.44     -1.93   0.02   16.98  19.05
8340  284611707     4.66  1.63     -1.91  -0.17   16.98  18.93
8341  284622010     4.65  1.75     -1.87  -0.32   16.99  18.85

[8342 rows x 7 columns]


In [8]:
if len(att_m1_fault5)>len(xkf1_m1_fault5):
    to_add=att_m1_fault5[len(xkf1_m1_fault5):]
    att_m1_fault5=att_m1_fault5[:len(xkf1_m1_fault5)]

if len(xkf1_m1_fault5)>len(att_m1_fault5):
    to_add=xkf1_m1_fault5[len(att_m1_fault5):]

for idx,i in enumerate(att_m1_fault5['Roll']):
    if(xkf1_m1_fault5['Roll'][idx] != i):
        att_m1_fault5['Roll'][idx] = ((att_m1_fault5['Roll'][idx] + xkf1_m1_fault5['Roll'][idx])/2)

for idx,i in enumerate(att_m1_fault5['Pitch']):
    if(xkf1_m1_fault5['Pitch'][idx] != i):
        att_m1_fault5['Pitch'][idx] = ((att_m1_fault5['Pitch'][idx] + xkf1_m1_fault5['Pitch'][idx])/2)

for idx,i in enumerate(att_m1_fault5['Yaw']):
    if(xkf1_m1_fault5['Yaw'][idx] != i):
        att_m1_fault5['Yaw'][idx] = ((att_m1_fault5['Yaw'][idx] + xkf1_m1_fault5['Yaw'][idx])/2)

att_m1_fault5=pd.concat([att_m1_fault5,to_add])

print(att_m1_fault5)

C:\Users\mgale\AppData\Local\Temp\ipykernel_22288\3051079878.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m1_fault5['Roll'][idx] = ((att_m1_fault5['Roll'][idx] + xkf1_m1_fault5['Roll'][idx])/2)
C:\Users\mgale\AppData\Local\Temp\ipykernel_22288\3051079878.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m1_fault5['Pitch'][idx] = ((att_m1_fault5['Pitch'][idx] + xkf1_m1_fault5['Pitch'][idx])/2)
C:\Users\mgale\AppData\Local\Temp\ipykernel_22288\3051079878.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://panda

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     192490624    -0.06  1.25      0.00   0.19   19.50  19.58
1     192500805    -0.07  1.34      0.00   0.27   19.50  19.60
2     192512203    -0.08  1.42      0.00   0.24   19.50  19.58
3     192523184    -0.08  1.48      0.00   0.17   19.50  19.57
4     192533737    -0.09  1.47      0.00   0.11   19.50  19.56
...         ...      ...   ...       ...    ...     ...    ...
8337  284578343     4.71  1.35     -2.01   0.18   16.97  19.19
8338  284588791     4.69  1.39     -1.96   0.11   16.98  19.12
8339  284599135     4.67  1.44     -1.93   0.02   16.98  19.05
8340  284611707     4.66  1.63     -1.91  -0.17   16.98  18.93
8341  284622010     4.65  1.75     -1.87  -0.32   16.99  18.85

[8342 rows x 7 columns]


In [9]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m1_fault5 = pd.read_csv("ESC_0.csv")
esc_0_m1_fault5 = esc_0_m1_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m1_fault5 = esc_0_m1_fault5[((esc_0_m1_fault5['TimeUS'] >= min) & (esc_0_m1_fault5['TimeUS'] <= max))]
esc_0_m1_fault5=esc_0_m1_fault5.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m1_fault5 = pd.read_csv("ESC_1.csv")
esc_1_m1_fault5 = esc_1_m1_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m1_fault5 = esc_1_m1_fault5[((esc_1_m1_fault5['TimeUS'] >= min) & (esc_1_m1_fault5['TimeUS'] <= max))]
esc_1_m1_fault5=esc_1_m1_fault5.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m1_fault5 = pd.read_csv("ESC_2.csv")
esc_2_m1_fault5 = esc_2_m1_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m1_fault5 = esc_2_m1_fault5[((esc_2_m1_fault5['TimeUS'] >= min) & (esc_2_m1_fault5['TimeUS'] <= max))]
esc_2_m1_fault5=esc_2_m1_fault5.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m1_fault5 = pd.read_csv("ESC_3.csv")
esc_3_m1_fault5 = esc_3_m1_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m1_fault5 = esc_3_m1_fault5[((esc_3_m1_fault5['TimeUS'] >= min) & (esc_3_m1_fault5['TimeUS'] <= max))]
esc_3_m1_fault5=esc_3_m1_fault5.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m1_fault5 = pd.read_csv("ESC_4.csv")
esc_4_m1_fault5 = esc_4_m1_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m1_fault5 = esc_4_m1_fault5[((esc_4_m1_fault5['TimeUS'] >= min) & (esc_4_m1_fault5['TimeUS'] <= max))]
esc_4_m1_fault5=esc_4_m1_fault5.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m1_fault5 = pd.read_csv("ESC_5.csv")
esc_5_m1_fault5 = esc_5_m1_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m1_fault5 = esc_5_m1_fault5[((esc_5_m1_fault5['TimeUS'] >= min) & (esc_5_m1_fault5['TimeUS'] <= max))]
esc_5_m1_fault5=esc_5_m1_fault5.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m1_fault5 = pd.read_csv("IMU_0.csv")
imu_0_m1_fault5 = imu_0_m1_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m1_fault5 = imu_0_m1_fault5[((imu_0_m1_fault5['TimeUS'] >= min) & (imu_0_m1_fault5['TimeUS'] <= max))]

imu_1_m1_fault5 = pd.read_csv("IMU_1.csv")
imu_1_m1_fault5 = imu_1_m1_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m1_fault5 = imu_1_m1_fault5[((imu_1_m1_fault5['TimeUS'] >= min) & (imu_1_m1_fault5['TimeUS'] <= max))]

imu_2_m1_fault5 = pd.read_csv("IMU_2.csv")
imu_2_m1_fault5 = imu_2_m1_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m1_fault5 = imu_2_m1_fault5[((imu_2_m1_fault5['TimeUS'] >= min) & (imu_2_m1_fault5['TimeUS'] <= max))]

imu_m1_fault5 = pd.concat((imu_0_m1_fault5, imu_1_m1_fault5, imu_2_m1_fault5))
imu_m1_fault5=imu_m1_fault5.groupby(imu_m1_fault5.TimeUS, as_index=False).mean()

display(imu_m1_fault5)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,192482549,-0.153982,-0.186241,-0.034864,-0.018558,0.074923,-9.849212
1,192485022,0.040070,-0.083952,-0.024396,-0.255245,0.288816,-10.166567
2,192488007,0.166860,0.142534,0.008056,-0.310567,0.183615,-10.046817
3,192490421,0.042349,0.164015,0.020348,-0.075254,-0.093423,-9.671370
4,192492994,-0.168712,-0.092433,-0.014593,0.184634,-0.077262,-9.611569
...,...,...,...,...,...,...,...
31686,284611544,0.107048,-0.187812,-0.135075,0.013124,-0.374169,-12.181003
31687,284614408,0.148649,-0.260531,-0.140879,-0.119346,-0.272562,-12.215220
31688,284617009,0.197476,-0.334106,-0.137907,-0.131811,-0.185854,-12.369360
31689,284619773,0.226640,-0.287556,-0.125083,-0.290287,-0.079025,-12.356240


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m1_fault5 = pd.merge_ordered(imu_m1_fault5,att_m1_fault5)
m1_fault5=m1_fault5.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m1_fault5 = pd.merge_ordered(m1_fault5, esc_0_m1_fault5)
m1_fault5 = pd.merge_ordered(m1_fault5, esc_1_m1_fault5)
m1_fault5 = pd.merge_ordered(m1_fault5, esc_2_m1_fault5)
m1_fault5 = pd.merge_ordered(m1_fault5, esc_3_m1_fault5)
m1_fault5 = pd.merge_ordered(m1_fault5, esc_4_m1_fault5)
m1_fault5 = pd.merge_ordered(m1_fault5, esc_5_m1_fault5)
m1_fault5=m1_fault5.fillna(method="ffill").fillna(method="bfill")
m1_fault5 = pd.merge_ordered(m1_fault5, rcou_m1_fault5)
m1_fault5=m1_fault5.fillna(method="ffill").fillna(method="bfill")

In [12]:
m1_fault5["TimeUS"] = m1_fault5["TimeUS"] - m1_fault5.iloc[0]["TimeUS"]
m1_fault5["TimeUS"] = m1_fault5["TimeUS"].astype(int)
m1_fault5["Guasto"] = guasto

In [13]:
last_cell = m1_fault5.iloc[-1, m1_fault5.columns.get_loc('TimeUS')]
print(last_cell)

92142228


In [14]:
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)

In [15]:
df = pd.DataFrame(to_be, columns = ['TimeUS'])

print(df)

         TimeUS
0             0
1          2857
2          5714
3          8571
4         11428
...         ...
32247  92129679
32248  92132536
32249  92135393
32250  92138250
32251  92141107

[32252 rows x 1 columns]


In [16]:
m1_fault5_final = pd.merge_ordered(m1_fault5,df)
m1_fault5_final=m1_fault5_final.fillna(method="ffill").fillna(method="bfill")
print(m1_fault5_final)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0             0 -0.153982 -0.186241 -0.034864 -0.018558  0.074923  -9.849212   
1          1809 -0.153982 -0.186241 -0.034864 -0.018558  0.074923  -9.849212   
2          2857 -0.153982 -0.186241 -0.034864 -0.018558  0.074923  -9.849212   
3          4282  0.040070 -0.083952 -0.024396 -0.255245  0.288816 -10.166567   
4          4470  0.040070 -0.083952 -0.024396 -0.255245  0.288816 -10.166567   
...         ...       ...       ...       ...       ...       ...        ...   
84723  92139289  0.226640 -0.287556 -0.125083 -0.290287 -0.079025 -12.356240   
84724  92141107  0.226640 -0.287556 -0.125083 -0.290287 -0.079025 -12.356240   
84725  92141161  0.184345 -0.221365 -0.115100 -0.387835 -0.135668 -12.431217   
84726  92141270  0.184345 -0.221365 -0.115100 -0.387835 -0.135668 -12.431217   
84727  92142228  0.184345 -0.221365 -0.115100 -0.387835 -0.135668 -12.431217   

       DesRoll  Roll  DesPitch  ...  Cu

In [17]:
m1_fault5_final = m1_fault5_final[m1_fault5_final.TimeUS.isin(to_be)]

In [18]:
m1_fault5_final

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,0,-0.153982,-0.186241,-0.034864,-0.018558,0.074923,-9.849212,-0.06,1.25,0.00,...,2.63,3324.0,2.02,1373.0,1423.0,1430.0,1365.0,1401.0,1395.0,1.0
2,2857,-0.153982,-0.186241,-0.034864,-0.018558,0.074923,-9.849212,-0.06,1.25,0.00,...,2.63,3324.0,2.02,1373.0,1423.0,1430.0,1365.0,1401.0,1395.0,1.0
5,5714,0.040070,-0.083952,-0.024396,-0.255245,0.288816,-10.166567,-0.06,1.25,0.00,...,2.63,3324.0,2.02,1373.0,1423.0,1430.0,1365.0,1401.0,1395.0,1.0
7,8571,0.166860,0.142534,0.008056,-0.310567,0.183615,-10.046817,-0.06,1.25,0.00,...,2.63,3324.0,2.02,1373.0,1423.0,1430.0,1365.0,1401.0,1395.0,1.0
10,11428,0.042349,0.164015,0.020348,-0.075254,-0.093423,-9.671370,-0.06,1.25,0.00,...,2.63,3324.0,2.02,1373.0,1423.0,1430.0,1365.0,1401.0,1395.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84713,92129679,0.118813,-0.206151,-0.138159,0.113017,-0.499642,-12.265113,4.67,1.44,-1.93,...,2.18,5743.0,3.56,1557.0,1458.0,1561.0,1453.0,1425.0,1583.0,1.0
84717,92132536,0.107048,-0.187812,-0.135075,0.013124,-0.374169,-12.181003,4.66,1.63,-1.91,...,2.18,5743.0,3.56,1560.0,1427.0,1576.0,1407.0,1464.0,1529.0,1.0
84719,92135393,0.148649,-0.260531,-0.140879,-0.119346,-0.272562,-12.215220,4.66,1.63,-1.91,...,2.18,5743.0,3.56,1560.0,1427.0,1576.0,1407.0,1464.0,1529.0,1.0
84721,92138250,0.197476,-0.334106,-0.137907,-0.131811,-0.185854,-12.369360,4.66,1.63,-1.91,...,2.18,5743.0,3.56,1560.0,1427.0,1576.0,1407.0,1464.0,1529.0,1.0


In [19]:
from datetime import timedelta
m1_fault5_final=m1_fault5_final.reset_index(drop=True)
m1_fault5_final['TimeUS'] = pd.to_datetime(m1_fault5_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m1_fault5_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,-0.153982,-0.186241,-0.034864,-0.018558,0.074923,-9.849212,-0.06,1.25,0.00,...,2.63,3324.0,2.02,1373.0,1423.0,1430.0,1365.0,1401.0,1395.0,1.0
1,00:00:00.002857,-0.153982,-0.186241,-0.034864,-0.018558,0.074923,-9.849212,-0.06,1.25,0.00,...,2.63,3324.0,2.02,1373.0,1423.0,1430.0,1365.0,1401.0,1395.0,1.0
2,00:00:00.005714,0.040070,-0.083952,-0.024396,-0.255245,0.288816,-10.166567,-0.06,1.25,0.00,...,2.63,3324.0,2.02,1373.0,1423.0,1430.0,1365.0,1401.0,1395.0,1.0
3,00:00:00.008571,0.166860,0.142534,0.008056,-0.310567,0.183615,-10.046817,-0.06,1.25,0.00,...,2.63,3324.0,2.02,1373.0,1423.0,1430.0,1365.0,1401.0,1395.0,1.0
4,00:00:00.011428,0.042349,0.164015,0.020348,-0.075254,-0.093423,-9.671370,-0.06,1.25,0.00,...,2.63,3324.0,2.02,1373.0,1423.0,1430.0,1365.0,1401.0,1395.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32247,00:01:32.129679,0.118813,-0.206151,-0.138159,0.113017,-0.499642,-12.265113,4.67,1.44,-1.93,...,2.18,5743.0,3.56,1557.0,1458.0,1561.0,1453.0,1425.0,1583.0,1.0
32248,00:01:32.132536,0.107048,-0.187812,-0.135075,0.013124,-0.374169,-12.181003,4.66,1.63,-1.91,...,2.18,5743.0,3.56,1560.0,1427.0,1576.0,1407.0,1464.0,1529.0,1.0
32249,00:01:32.135393,0.148649,-0.260531,-0.140879,-0.119346,-0.272562,-12.215220,4.66,1.63,-1.91,...,2.18,5743.0,3.56,1560.0,1427.0,1576.0,1407.0,1464.0,1529.0,1.0
32250,00:01:32.138250,0.197476,-0.334106,-0.137907,-0.131811,-0.185854,-12.369360,4.66,1.63,-1.91,...,2.18,5743.0,3.56,1560.0,1427.0,1576.0,1407.0,1464.0,1529.0,1.0


## Analisi nel tempo e in frequenza

In [20]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [21]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m1_fault5_final)/350)):
        V1=m1_fault5_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab=tab.append(val,ignore_index=True)
    return tab

In [22]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto
display(df_merged)

d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_22288\1704534960.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_22288\1704534960.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,-0.018465,0.009846,2.594661,0.100790,60.0,0.0,4.590620,4.396057,-0.006336,...,7482.852006,3.429858,0.518868,-0.270225,3.504470,130.0,131.0,16.649461,14.084934,1
1,1.0,-0.018733,0.009843,2.584391,0.100827,58.0,0.0,4.548868,4.256903,-0.006562,...,6904.270235,3.428693,0.517867,-0.270357,3.503189,126.0,127.0,16.252153,13.787651,1
2,2.0,-0.019001,0.009840,2.574363,0.100864,60.0,0.0,4.934244,4.117667,-0.006786,...,6328.604459,3.427535,0.516869,-0.270438,3.501914,138.0,139.0,15.837539,13.485232,1
3,3.0,-0.019095,0.009816,2.586751,0.100760,56.0,0.0,4.866304,4.039080,-0.006824,...,5761.737957,3.426384,0.515874,-0.270469,3.500646,150.0,151.0,15.407816,13.178185,1
4,4.0,-0.019413,0.009824,2.567005,0.100862,51.0,0.0,4.536598,3.880765,-0.006944,...,5209.024098,3.425239,0.514882,-0.270449,3.499385,134.0,135.0,14.965120,12.867007,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,87.0,-0.039511,0.010097,1.781742,0.107866,101.0,76.0,5.814520,5.177905,-0.010776,...,5135.676023,3.409087,0.429666,0.137237,3.471391,140.0,142.0,14.978194,13.903976,1
88,88.0,-0.039506,0.010074,1.792549,0.107758,99.0,78.0,5.922625,4.498922,-0.010599,...,5187.516900,3.409613,0.428806,0.144076,3.471785,142.0,144.0,14.602500,13.377862,1
89,89.0,-0.039538,0.010052,1.803404,0.107666,87.0,66.0,6.002336,4.950231,-0.010517,...,5228.838455,3.410136,0.427950,0.150922,3.472176,142.0,144.0,14.136354,12.849370,1
90,90.0,-0.039668,0.010036,1.809067,0.107642,93.0,72.0,6.027243,5.231174,-0.010595,...,5259.239789,3.410658,0.427098,0.157776,3.472566,140.0,143.0,13.586418,12.429978,1


In [23]:
path_file = path_file.replace(r"M1\FAULT_M1_5", "")
os.chdir(path_file)
df_merged.to_csv('m1_fault5.csv', index=False)
print(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\
